# Artificial Market Simulator
This notebook demonstrates:
- Simulating market order flow and prices with the generalized propagator model
- Reconstructing proxy metaorders from trade-by-trade data


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from scipy.stats import powerlaw
from scipy.linalg import toeplitz, cholesky
from numba import njit


## Simulation Utilities

In [6]:
def mu_q(mu_1, q, l):
    return mu_1 + l * np.log(q)

def beta_q(beta_1, q, lprime):
    return min(max(beta_1 - lprime*np.log(q), 0.0), 0.8)

def gamma_q(mean_q, sigma_q):
    return max(1, np.random.lognormal(mean=mean_q, sigma=sigma_q))

def generate_poisson_times(n, nu):
    inter_event_times = np.random.exponential(scale=1/nu, size=n)
    event_times = np.cumsum(inter_event_times).astype(int)
    for i in range(1, len(event_times)):
        if event_times[i] <= event_times[i-1]:
            event_times[i] = event_times[i-1] + 1
    return event_times


# Code to generate sign series with an autocorrelation E[eps_t * eps_{t+tau}] = \Gamma tau^{-\gamma} where 
# gamma and Gamma are controlled

def generate_sign_series(N, gamma, Gamma, T_cut=300, seed=None):
    if seed is not None:
        np.random.seed(seed)
    acf_eps = np.clip(Gamma * np.arange(1, N+1)**(-gamma), -1, 1)
    acf_eps[acf_eps < 0] = 0
    acf_eps[:T_cut] = acf_eps[:T_cut]
    acf_x = np.sin((np.pi / 2) * acf_eps)
    cov = toeplitz(acf_x)
    L = cholesky(cov, lower=True)
    x = L @ np.random.normal(size=N)
    eps = np.sign(x)
    eps[eps == 0] = 1
    return eps


## Market Simulation Function

In [11]:
# This function can be readily adapted to Numba for performance gains.

def simulate_market(nb_iter, corr, mu_1, beta_1, mean_q, sigma_q, l, lprime,
                    phi_tilde, delta_meta_start):
    meta_times = generate_poisson_times(nb_iter, 1/delta_meta_start)
    vols = [round(gamma_q(mean_q, sigma_q), 2) for _ in range(nb_iter)]
    if corr:
        signs = generate_sign_series(nb_iter, 0.5, 0.1 if l == 0 else 0.05, T_cut=500)
        l = l/2 if l > 0 else l
    else:
        signs = 2*np.random.randint(0, 2, nb_iter) - 1
    mus = np.array([mu_q(mu_1, v, l) for v in vols])
    betas = np.array([beta_q(beta_1, v, lprime) for v in vols])
    sizes = (1 / powerlaw.rvs(mus)).astype(int) + 1
    child_orders = []
    for i in range(nb_iter):
        vol, sign, beta, s, t_start = vols[i], signs[i], betas[i], sizes[i], meta_times[i]
        inter_arrival = np.random.exponential(scale=phi_tilde, size=s-1)
        exec_times = np.sort(t_start + np.cumsum(np.insert(inter_arrival, 0, 0)))
        for rank, t_exec in enumerate(exec_times, start=1):
            child_orders.append((t_exec, sign, vol, t_start, rank, beta))
    child_orders.sort(key=lambda x: x[0])
    child_orders = np.array(child_orders, dtype=object)
    t_execs = child_orders[:, 0].astype(int)
    signs_arr = child_orders[:, 1].astype(int)
    vols_arr = child_orders[:, 2].astype(float)
    t_start_arr = child_orders[:, 3].astype(int)
    rank_arr = child_orders[:, 4].astype(int)
    betas_arr = child_orders[:, 5].astype(float)
    prices = np.zeros(len(child_orders))
    tau0 = np.mean(np.diff(t_execs))
    for i in range(len(child_orders)):
        past = t_execs[:i] < t_execs[i]
        if not np.any(past): continue
        idx_past = np.where(past)[0]
        t_prime = (t_execs[idx_past] - t_start_arr[idx_past]) / phi_tilde
        dt = t_execs[i] - t_execs[idx_past]
        scale = (vols_arr[idx_past]**0.5) / (t_prime+tau0)**(0.5 - betas_arr[idx_past])
        decay = (tau0 / (dt + tau0))**betas_arr[idx_past]
        prices[i] = np.sum(signs_arr[idx_past] * scale * decay)
    base_time = datetime(2024,1,1)
    exec_time_str = [base_time + timedelta(milliseconds=int(ms)) for ms in t_execs]
    start_time_str = [base_time + timedelta(milliseconds=int(ms)) for ms in t_start_arr]
    df = pd.DataFrame({
        't_exec': t_execs,
        'price': prices,
        'qty': vols_arr,
        't_start': t_start_arr,
        'child': rank_arr,
        'sign': signs_arr,
        'time': [ts.strftime("%H:%M:%S,%f")[:-3] for ts in exec_time_str],
        'start_time': [ts.strftime("%H:%M:%S,%f")[:-3] for ts in start_time_str],
        'beta': betas_arr
    })
    return df


## A new mapping function for creating synthetic metaorders 

In [18]:
@njit
def _u():
    r = np.random.random()
    return min(max(r, 1e-12), 1 - 1e-12)

@njit
def draw_powerlaw_size(alpha=1.5, size_max=2000):
    s = int(_u() ** (-1.0 / alpha)) + 1
    return min(s, size_max)

@njit
def draw_exponential_gap(phi):
    return 1 + int(-phi * np.log(_u()))

@njit
def generate_synthetic_metaorders_ids(n_rows, phi_tilde=200, alpha=1.5, max_size=3000):
    ids = np.zeros(n_rows, dtype=np.int64)
    id_meta, i = 1, 0
    while i < n_rows:
        size = draw_powerlaw_size(alpha, max_size)
        j, k = i, 0
        while k < size and j < n_rows:
            ids[j] = id_meta
            j += draw_exponential_gap(phi_tilde)
            k += 1
        id_meta += 1
        i += 1
        while i < n_rows and ids[i] != 0:
            i += 1
    return ids

def map_meta(df, phi_tilde=200, alpha=1.5, max_size=3000):
    n_rows = len(df)
    ids = generate_synthetic_metaorders_ids(n_rows, phi_tilde, alpha, max_size)
    df = df.copy()
    df['synthetic_id_meta'] = ids
    return df


## Example Usage

In [20]:
df = simulate_market(500, corr=True, mu_1=1.5, beta_1=0.25,
                     mean_q=3, sigma_q=1, l=0, lprime=0, phi_tilde=500, delta_meta_start=200)
df_meta = map_meta(df, phi_tilde=200)

df_meta.head()


,t_exec,price,qty,t_start,child,sign,time,start_time,beta,synthetic_id_meta
0,427,0.000000,12.50,427,1,-1,"00:00:00,427","00:00:00,427",0.25,1
1,445,-1.026345,12.50,427,2,-1,"00:00:00,445","00:00:00,427",0.25,2
2,874,-1.453054,2.27,874,1,1,"00:00:00,874","00:00:00,874",0.25,3
3,944,-1.006614,14.74,944,1,1,"00:00:00,944","00:00:00,944",0.25,4
4,1188,-0.093320,98.26,1188,1,-1,"00:00:01,188","00:00:01,188",0.25,5
